In [1]:
import numpy as np
import random
from copy import deepcopy
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import math,time,sys
from matplotlib import pyplot
import pandas as pd
from datetime import datetime
from functools import partial
import seaborn as sns 
from sklearn.metrics import roc_auc_score
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

MaxIter = 1
pop_size = 8
omega = 0.99


def initialise(partCount, dim, trainX, testX, trainy, testy):    
    population=np.zeros((partCount,dim))
    minn = 1
    maxx = math.floor(0.5*dim)
    
    if maxx<minn:
        maxx = minn + 1
        #not(c[i].all())
    
    for i in range(partCount):
        random.seed(i**3 + 10 + time.time() ) 
        no = random.randint(minn,maxx)
        if no == 0:
            no = 1
        random.seed(time.time()+ 100)
        pos = random.sample(range(0,dim-1),no)
        for j in pos:
            population[i][j]=1
            
    return population

def fitness(agent, trainX, testX, trainy, testy):
    # print(agent)
    cols=np.flatnonzero(agent)
    # print(cols)
    val=1
    if np.shape(cols)[0]==0:
        return val    
    clf=KNeighborsClassifier(n_neighbors=5)
    #clf=MLPClassifier(alpha=0.001, hidden_layer_sizes=(1000,500,100),max_iter=2000,random_state=4)
    train_data=trainX[:,cols]
    test_data=testX[:,cols]
    clf.fit(train_data,trainy)
    val=1-clf.score(test_data,testy)

    #in case of multi objective  []
    set_cnt=sum(agent)
    set_cnt=set_cnt/np.shape(agent)[0]
    val=omega*val+(1-omega)*set_cnt
    return val

def test_accuracy(agent, trainX, testX, trainy, testy):
    cols=np.flatnonzero(agent)
    val=1
    if np.shape(cols)[0]==0:
        return val    
    # clf = RandomForestClassifier(n_estimators=300)
    #clf=MLPClassifier(alpha=0.001, hidden_layer_sizes=(1000,500,100),max_iter=2000,random_state=4)
    clf=KNeighborsClassifier(n_neighbors=5)
    # clf=MLPClassifier( alpha=0.01, max_iterno=1000) #hidden_layer_sizes=(1000,500,100)
    #cross=4
    #test_size=(1/cross)
    #X_train, X_test, y_train, y_test = train_test_split(trainX, trainy,  stratify=trainy,test_size=test_size)
    train_data=trainX[:,cols]
    test_data=testX[:,cols]
    clf.fit(train_data,trainy)
    val=clf.score(test_data,testy)
    return val

def onecnt(agent):
    return sum(agent)

def sigmoid(gamma):
    if gamma < 0:
        return 1 - 1/(1 + math.exp(gamma))
    else:
        return 1/(1 + math.exp(-gamma))
    
def GWO(dataset):
    df = pd.read_csv(dataset)
    a, b = np.shape(df)
    data = df.values[:,0:b-1]
    label = df.values[:,b-1]
    dimension = data.shape[1]
    
    cross = 5
    test_size = (1/cross)
    trainX, testX, trainy, testy = train_test_split(data, label,stratify=label ,test_size=test_size,random_state=(7+17*int(time.time()%1000)))
    clf=KNeighborsClassifier(n_neighbors=5)
    clf.fit(trainX,trainy)
    val=clf.score(testX,testy)
    whole_accuracy = val
    print("Total Acc: ",val)
    
    pop = initialise(pop_size, dimension, trainX, testX, trainy, testy)
        
    
    for n in range(MaxIter):
        fit = []
        for i in range(pop_size):
            fit.append(fitness(pop[i], trainX, testX, trainy, testy))
        ind = np.argsort(fit)
        alpha = pop[ind[0]]
        alpha_fit = fit[ind[0]]
        beta = pop[ind[1]]
        beta_fit = pop[ind[1]]
        delta = pop[ind[2]]
        delta_fit = fit[ind[2]]
        
        a=2-n*((2)/MaxIter); # a decreases linearly fron 2 to 0
        
        # Update the Position of search agents including omegas
        for i in range(pop_size):
            for j in range (dimension):     
                           
                r1=random.random() # r1 is a random number in [0,1]
                r2=random.random() # r2 is a random number in [0,1]
                
                A1=2*a*r1-a; # Equation (3.3)
                C1=2*r2; # Equation (3.4)
                
                D_alpha=abs(C1*alpha[j]-pop[i,j]); # Equation (3.5)-part 1
                X1=alpha[j]-A1*D_alpha; # Equation (3.6)-part 1
                           
                r1=random.random()
                r2=random.random()
                
                A2=2*a*r1-a; # Equation (3.3)
                C2=2*r2; # Equation (3.4)
                
                D_beta=abs(C2*beta[j]-pop[i,j]); # Equation (3.5)-part 2
                X2=beta[j]-A2*D_beta; # Equation (3.6)-part 2       
                
                r1=random.random()
                r2=random.random() 
                
                A3=2*a*r1-a; # Equation (3.3)
                C3=2*r2; # Equation (3.4)
                
                D_delta=abs(C3*delta[j]-pop[i,j]); # Equation (3.5)-part 3
                X3=delta[j]-A3*D_delta; # Equation (3.5)-part 3             
                
                pop[i,j]=(X1+X2+X3)/3  # Equation (3.7)

                if (sigmoid(pop[i][j]) > random.random()):
                    pop[i][j] = 1
                else:
                    pop[i][j] = 0
        
    ind = np.argsort(fit)
    bestpop = pop[ind[0]]
    bestfit = fit[ind[0]]
    
    testAcc = test_accuracy(bestpop, trainX, testX, trainy, testy)
    featCnt = onecnt(bestpop)
    #print("best agent: ", bestpop)
    print("Test Accuracy: ", testAcc)
    print("#Features: ", featCnt)
            
    return testAcc, featCnt, bestpop

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
%%time
Ta,fc,bp = GWO('IITM_Spec128.csv')

Total Acc:  0.8784615384615385
Test Accuracy:  0.8815384615384615
#Features:  88.0
CPU times: user 823 ms, sys: 40 ms, total: 863 ms
Wall time: 891 ms


In [3]:
List = []

for i in range(0,len(bp)):
    if bp[i] == 1:
        List.append(i)
        
df_train = pd.read_csv("IITM_Spec128.csv")
y = (df_train['label'])
df_train1 = df_train[df_train.columns[List]]
l1 = []
for i in range(0,len(List)):
    l1.append(i)
    
df_train1.columns = l1

df_train1['label'] = y
df_train1

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,0,1,2,3,4,5,6,7,8,9,...,79,80,81,82,83,84,85,86,87,label
0,0.011445,0.005392,0.001609,0.000516,0.000685,0.001098,0.048293,1.747653,1.892785,0.583005,...,0.000175,0.000229,0.000238,0.000497,0.000488,0.000551,0.000330,0.000254,0.000186,Assamese
1,0.000934,0.000941,0.007122,0.038213,0.432158,0.604798,0.577700,0.092206,0.332166,0.875804,...,0.000465,0.000561,0.000966,0.000614,0.000420,0.000471,0.000441,0.000196,0.000147,Assamese
2,0.001019,0.001124,0.018744,0.107425,0.568368,1.121817,0.391567,0.099762,0.285366,0.906578,...,0.000173,0.000214,0.000318,0.000282,0.000150,0.000183,0.000190,0.000131,0.000048,Assamese
3,0.000962,0.002874,0.035911,0.188132,0.575486,0.781521,0.699432,0.158355,0.304569,0.600820,...,0.000221,0.000329,0.000366,0.000200,0.000219,0.000151,0.000180,0.000242,0.000305,Assamese
4,0.046121,0.050567,0.278968,1.727229,6.446747,8.184534,4.493108,2.688246,1.567698,1.897226,...,0.000907,0.000706,0.000615,0.000423,0.000264,0.000245,0.000159,0.000139,0.000419,Assamese
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3245,0.016221,0.010791,0.045113,0.374282,2.848596,5.531259,8.345696,16.297132,16.385887,20.768005,...,0.003642,0.002945,0.002165,0.001634,0.001306,0.000754,0.000368,0.000104,0.000010,Telugu
3246,0.070613,0.016239,0.052733,0.622141,3.334931,6.250635,11.021016,23.813210,15.667769,4.889742,...,0.007207,0.006048,0.005397,0.003770,0.003318,0.002020,0.000938,0.000227,0.000011,Telugu
3247,0.358309,0.153024,0.153383,1.099469,4.843939,7.587422,9.914358,12.931074,5.615302,2.793209,...,0.009679,0.008005,0.008298,0.008782,0.006930,0.005167,0.001888,0.000514,0.000028,Telugu
3248,0.193823,0.065367,0.135263,0.653783,3.271528,5.483109,7.623892,15.562514,15.815204,9.590445,...,0.011770,0.009638,0.011048,0.010172,0.007675,0.005179,0.002411,0.000744,0.000040,Telugu


In [4]:
df_train1.to_csv('IITM_BGWO.csv')